In [1]:
from src.models import Sequential
from src.layers import *
from src.activation_functions import *
from src.loss_functions import *

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
the_image = np.random.random((1, 1, 28, 28))

In [3]:
# weights = np.random.normal(0, stddev, (n_inputs, n_outputs, filter_size, filter_size)).astype(np.float32)

In [4]:
img = np.asarray(
    [
        [
            [
                [2, 2, 0, 0, 0],
                [2, 0, 1, 0, 1],
                [2, 0, 1, 2, 1],
                [2, 0, 0, 1, 0],
                [0, 2, 1, 0, 1],
            ],
            [
                [2, 1, 1, 2, 0],
                [0, 2, 1, 2, 1],
                [2, 2, 1, 0, 2],
                [2, 2, 2, 2, 0],
                [1, 2, 0, 2, 1],
            ],
            [
                [0, 1, 1, 0, 0],
                [0, 0, 1, 2, 0],
                [2, 0, 1, 1, 2],
                [1, 1, 2, 1, 2],
                [0, 2, 1, 0, 1],
            ],
        ],
        [
            [
                [2, 2, 0, 0, 0],
                [2, 0, 1, 0, 1],
                [2, 0, 1, 2, 1],
                [2, 0, 0, 1, 0],
                [0, 2, 1, 0, 1],
            ],
            [
                [2, 1, 1, 2, 0],
                [0, 2, 1, 2, 1],
                [2, 2, 1, 0, 2],
                [2, 2, 2, 2, 0],
                [1, 2, 0, 2, 1],
            ],
            [
                [0, 1, 1, 0, 0],
                [0, 0, 1, 2, 0],
                [2, 0, 1, 1, 2],
                [1, 1, 2, 1, 2],
                [0, 2, 1, 0, 1],
            ],
        ],
    ]
)

the_image = np.pad(img, ((0, 0), (0, 0), (1, 1), (1, 1)))


the_image.shape

(2, 3, 7, 7)

In [5]:
weights = np.asarray(
    [
        [
            [[1, 0, 0], [0, 0, -1], [0, 0, -1]],
            [[1, 1, 0], [-1, 1, -1], [1, 1, 1]],
            [[0, -1, 1], [1, -1, -1], [0, -1, 1]],
        ],
        [
            [[0, -1, 1], [0, 0, -1], [1, 0, -1]],
            [[1, 1, -1], [0, 0, 0], [-1, 0, -1]],
            [[0, 0, 1], [0, 0, -1], [0, 1, -1]],
        ],
    ]
)

weights.shape

(2, 3, 3, 3)

In [6]:
import torch

conv = torch.nn.Conv2d(
    in_channels=3,
    out_channels=2,
    kernel_size=3,
    bias=False,
    stride=1,
    padding_mode="zeros",
    padding=1,
)

In [7]:
# conv.forward(torch.from_numpy(the_image).float())

In [8]:
x_tensor = torch.from_numpy(the_image.astype(np.float32))
x_tensor.requires_grad = True
conv.weight = torch.nn.Parameter(torch.from_numpy(weights.astype(np.float32)))

In [9]:
out_torch = conv.forward(x_tensor)
out_torch

tensor([[[[ 0.,  2.,  4.,  3.,  3.,  2.,  0.],
          [-6.,  0., -1.,  4.,  3.,  1.,  1.],
          [ 0.,  3.,  9., -2.,  4.,  3.,  1.],
          [-5.,  2., 11.,  3.,  1.,  4.,  2.],
          [-2.,  1.,  4.,  1.,  5.,  1.,  6.],
          [ 0., -3.,  4.,  0.,  5., -1.,  0.],
          [ 0.,  3.,  2.,  3.,  4.,  2.,  2.]],

         [[-4., -4., -1.,  0., -1., -2.,  0.],
          [-4., -5., -2., -5.,  0., -2.,  0.],
          [-8.,  2., -4., -6., -2.,  6., -1.],
          [-7., -6., -2., -5., -3.,  3.,  1.],
          [-2., -9.,  2.,  2., -4.,  0.,  2.],
          [ 1., -5.,  2.,  4.,  3.,  2.,  0.],
          [-1.,  3.,  3., -1.,  3.,  2.,  1.]]],


        [[[ 0.,  2.,  4.,  3.,  3.,  2.,  0.],
          [-6.,  0., -1.,  4.,  3.,  1.,  1.],
          [ 0.,  3.,  9., -2.,  4.,  3.,  1.],
          [-5.,  2., 11.,  3.,  1.,  4.,  2.],
          [-2.,  1.,  4.,  1.,  5.,  1.,  6.],
          [ 0., -3.,  4.,  0.,  5., -1.,  0.],
          [ 0.,  3.,  2.,  3.,  4.,  2.,  2.]],

     

In [10]:
output = np.zeros((2, 2, 7, 7))
ww = np.einsum("oifg -> iofg", weights).astype(np.float32)

the_image2 = np.pad(the_image, ((0, 0), (0, 0), (1, 1), (1, 1)), constant_values=0)

# for b in range(the_image.shape[0]):
for ot in range(2):
    for h in range(output.shape[2]):
        for w in range(output.shape[3]):
            output[:, ot, h, w] = np.sum(
                the_image2[:, :, h : h + 3, w : w + 3] * ww[:, ot], axis=(1, 2, 3)
            )

In [11]:
output

array([[[[ 0.,  2.,  4.,  3.,  3.,  2.,  0.],
         [-6.,  0., -1.,  4.,  3.,  1.,  1.],
         [ 0.,  3.,  9., -2.,  4.,  3.,  1.],
         [-5.,  2., 11.,  3.,  1.,  4.,  2.],
         [-2.,  1.,  4.,  1.,  5.,  1.,  6.],
         [ 0., -3.,  4.,  0.,  5., -1.,  0.],
         [ 0.,  3.,  2.,  3.,  4.,  2.,  2.]],

        [[-4., -4., -1.,  0., -1., -2.,  0.],
         [-4., -5., -2., -5.,  0., -2.,  0.],
         [-8.,  2., -4., -6., -2.,  6., -1.],
         [-7., -6., -2., -5., -3.,  3.,  1.],
         [-2., -9.,  2.,  2., -4.,  0.,  2.],
         [ 1., -5.,  2.,  4.,  3.,  2.,  0.],
         [-1.,  3.,  3., -1.,  3.,  2.,  1.]]],


       [[[ 0.,  2.,  4.,  3.,  3.,  2.,  0.],
         [-6.,  0., -1.,  4.,  3.,  1.,  1.],
         [ 0.,  3.,  9., -2.,  4.,  3.,  1.],
         [-5.,  2., 11.,  3.,  1.,  4.,  2.],
         [-2.,  1.,  4.,  1.,  5.,  1.,  6.],
         [ 0., -3.,  4.,  0.,  5., -1.,  0.],
         [ 0.,  3.,  2.,  3.,  4.,  2.,  2.]],

        [[-4., -4., -1.,  

In [12]:
loss = out_torch.sum()
loss.backward()
loss

tensor(70., grad_fn=<SumBackward0>)

In [13]:
x_tensor.grad

tensor([[[[ 0., -1., -1., -1., -1., -1., -2.],
          [ 1., -2., -2., -2., -2., -2., -4.],
          [ 1., -2., -2., -2., -2., -2., -4.],
          [ 1., -2., -2., -2., -2., -2., -4.],
          [ 1., -2., -2., -2., -2., -2., -4.],
          [ 1., -2., -2., -2., -2., -2., -4.],
          [ 1., -3., -3., -3., -3., -3., -4.]],

         [[ 4.,  2.,  2.,  2.,  2.,  2.,  1.],
          [ 5.,  3.,  3.,  3.,  3.,  3.,  2.],
          [ 5.,  3.,  3.,  3.,  3.,  3.,  2.],
          [ 5.,  3.,  3.,  3.,  3.,  3.,  2.],
          [ 5.,  3.,  3.,  3.,  3.,  3.,  2.],
          [ 5.,  3.,  3.,  3.,  3.,  3.,  2.],
          [ 1.,  0.,  0.,  0.,  0.,  0.,  1.]],

         [[-1., -1., -1., -1., -1., -1., -2.],
          [-1., -1., -1., -1., -1., -1., -2.],
          [-1., -1., -1., -1., -1., -1., -2.],
          [-1., -1., -1., -1., -1., -1., -2.],
          [-1., -1., -1., -1., -1., -1., -2.],
          [-1., -1., -1., -1., -1., -1., -2.],
          [ 0., -2., -2., -2., -2., -2., -3.]]],


     

In [14]:
the_image.shape

(2, 3, 7, 7)

In [15]:
error.shape

NameError: name 'error' is not defined

In [ ]:
error

In [ ]:
da = np.zeros_like(the_image)
da

In [ ]:
output.shape

In [ ]:
da = np.zeros_like(the_image).astype(np.float32)
error = np.ones_like(the_image)
error_padded = np.pad(error, ((0, 0), (0, 0), (1, 1), (1, 1)), constant_values=0)

for b in range(2):
    for i in range(3):
        for o in range(2):
            for h in range(7):
                for w in range(7):
                    da[b, i, h, w] += np.sum(
                        ww[i, o] * error_padded[b, o, h : h + 3, w : w + 3]
                    )

In [ ]:
da

In [ ]:
conv.weight.grad

In [98]:
da = np.zeros(shape=(1, 1, 6, 5)).astype(np.float32)
da

array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)

In [104]:
da.shape

(1, 1, 6, 5)

In [99]:
error = np.asarray(
    [
        [
            [
                [0.0, 0, 0, 0, 0.0],
                [0.0, 1, 1, 1, 0.0],
                [0.0, 1, 1, 1, 0.0],
                [0.0, 1, 1, 1, 0.0],
                [0.0, 1, 1, 1, 0.0],
                [0.0, 0, 0, 0, 0.0],
            ]
        ]
    ]
)
error

array([[[[0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0.],
         [0., 1., 1., 1., 0.],
         [0., 1., 1., 1., 0.],
         [0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0.]]]])

In [100]:
filter = np.asarray([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
filter

array([[-1,  0,  1],
       [-2,  0,  2],
       [-1,  0,  1]])

In [101]:
pad = (da.shape[2] - error.shape[2]) // 2 + 1
pad

1

In [102]:
print(pad)
error_padded = np.pad(
    error, ((0, 0), (0, 0), (pad, pad), (pad, pad)), constant_values=0
)

for i in range(1):
    for o in range(1):
        for h in range(6):
            for w in range(5):
                da[:, i, h, w] += np.sum(
                    filter * error_padded[:, o, h : h + 3, w : w + 3]
                )

1


In [103]:
da

array([[[[ 1.,  1.,  0., -1., -1.],
         [ 3.,  3.,  0., -3., -3.],
         [ 4.,  4.,  0., -4., -4.],
         [ 4.,  4.,  0., -4., -4.],
         [ 3.,  3.,  0., -3., -3.],
         [ 1.,  1.,  0., -1., -1.]]]], dtype=float32)

In [39]:
conv2d = Conv2D((3, 0, 0), 2, 3, 1, 0)
conv2d.weights = weights

In [ ]:
weights.shape

In [ ]:
conv2d.weights = np.einsum("oifg -> iofg", weights).astype(np.float32)

In [ ]:
conv2d.weights

In [ ]:
out = conv2d.forward(the_image)
out

In [ ]:
conv2d.backward(np.ones_like(out).astype(np.float32), 0.1)

In [ ]:
out.shape

In [ ]:
padded_out = np.pad(out, ((0, 0), (0, 0), (1, 1), (1, 1))).astype(np.float32)
dpo = np.zeros_like(the_image).astype(np.float32)

In [ ]:
dpo.shape

In [ ]:
weights.shape

In [ ]:
ttt = np.ones_like(out)
tew = np.sum(ttt, axis=(0, 2, 3))
tew

In [ ]:
padded_out = np.pad(out, ((0, 0), (0, 0), (1, 1), (1, 1))).astype(np.float32)
dpo = np.zeros_like(the_image).astype(np.float32)

# test_error = np.asarray([[[
#     [0, 0, 0, 0, 0],
#     [0, 1, 1, 1, 0],
#     [0, 1, 1, 1, 0],
#     [0, 1, 1, 1, 0],
#     [0, 1, 1, 1, 0],
#     [0, 0, 0, 0, 0]
# ]]])

# test_error = np.ones_like(test_error)

# error_padded = np.pad(test_error, ((0, 0), (0, 0), (1, 1), (1, 1)))
#
# error_padded = np.ones_like(out)
# pdd = (the_image.shape[2] - error_padded.shape[2]) // 2
# print(pdd)
# error_padded = np.pad(error_padded, ((0, 0), (0, 0), (pdd, pdd), (pdd, pdd)), constant_values=1)
# print(error_padded)
#
# for i in range(3):
#     for o in range(2):
#         for h in range(out.shape[2]):
#             for w in range(out.shape[3]):
#                 dpo[:, :, h:h+3,w:w+3] += ww * error_padded[:, :, h:h+3, w:w+3]

In [ ]:
dpo

In [ ]:
x_tensor.grad

In [ ]:
dw = np.zeros(shape=(3, 2, 3, 3))
error = np.ones_like(output)

In [ ]:
the_image.shape

In [ ]:
dw.shape

In [ ]:
for i in range(3):
    for o in range(2):
        for h in range(dw.shape[2]):
            for w in range(dw.shape[3]):
                dw[i, o, h, w] = np.sum(
                    the_image[:, i, h : h + 5, w : w + 5] * error[:, o]
                )

In [ ]:
dw

In [ ]:
import scipy

error = np.ones_like(output)

dw = np.zeros_like(conv2d.weights)

for b in range(2):
    for i in range(3):
        for o in range(2):
            dw[i, o] += scipy.signal.correlate2d(
                the_image[b][i], error[b][o], mode="valid"
            )

In [ ]:
dw

In [ ]:
conv.weight.grad

In [ ]:
print(error_padded.shape)

In [ ]:
the_image.shape

In [ ]:
dpo

In [ ]:
# error_conv = np.zeros_like(the_image)
#
# for c in range(out.shape[1]):
#     for h in range(out.shape[2]):
#         for w in range(out.shape[3]):
#             error_conv[:, :, h, w] = np.sum(
#                  weights[c, :] * error_padded[:, :, h:h+3, w:w+3]
#             )

In [ ]:
error_conv

In [ ]:
# dw = np.zeros_like(conv2d.weights).astype(np.float32)
# a_prev = np.pad(the_image, ((0,0), (0, 0), (1,1), (1,1))).astype(np.float32)
# err = np.ones_like(out)
#
# # for b in range(out.shape[0]):
# for i in range(dw.shape[0]):
#     for f in range(dw.shape[1]):
#         for h in range(dw.shape[2]):
#             for w in range(dw.shape[3]):
#                 dw[i, f] += a_prev[:, i, h:h+3, w:w+3]
#
# dw

In [ ]:
import scipy

dw = np.zeros_like(weights).astype(np.float32)
a_prev = np.pad(the_image, ((0, 0), (0, 0), (1, 1), (1, 1))).astype(np.float32)
err = np.ones_like(out)

for b in range(the_image.shape[0]):
    for o in range(dw.shape[0]):
        for i in range(dw.shape[1]):
            dw[o, i] += scipy.signal.correlate2d(
                the_image[b][i], err[b][o], mode="valid"
            )

dw

In [ ]:
dw = np.zeros_like(weights).astype(np.float32)
a_prev = np.pad(the_image, ((0, 0), (0, 0), (1, 1), (1, 1))).astype(np.float32)
err = np.ones_like(out)

for b in range(1):
    for c in range(2):
        for h in range(dw.shape[2]):
            for w in range(dw.shape[3]):
                dw[c] = a_prev[b, :, h : h + 3, w : w + 3] * err[b, c, h, w]

In [ ]:
dw

In [ ]:
the_image.shape

In [ ]:
import scipy

scipy.signal.correlate2d(the_image[0][2], err[0][0], mode="valid")

In [ ]:
# for c in range(out.shape[1]):
#     for h in range(out.shape[2]):
#         for w in range(out.shape[3]):
#             dw[:, c] += a_prev[:, :, h:h+3, w:w+3] * err[:, c, h, w]

In [ ]:
dw2 = np.zeros_like(conv2d.weights).astype(np.float32)

In [ ]:
conv.weight.grad

In [ ]:
dw.shape

In [ ]:
raise ValueError

In [ ]:
# # for b in range(output.shape[0]):
# for f in range(2):
#     for h in range(output.shape[2]):
#         for w in range(output.shape[3]):
#             output[:, f, h, w] = np.sum(the_image[:, :, h * 2:h * 2+3, w * 2:w * 2+3] * weights[f, :]) + bias[f]

In [ ]:
# output

In [ ]:
from src.layers import Conv2D

conv = Conv2D(2, 3, 3, 2, 1)

In [ ]:
conv.weights = weights.astype(np.float32)

In [ ]:
conv.bias = np.asarray([[1, 0]]).astype(np.float32)

In [ ]:
out_c = conv.forward(img)
# out_c

In [ ]:
conv.backward(out_c, 0.1)

In [ ]:
conv.weights

In [ ]:
test_error = np.asarray(
    [
        [
            [0, 0, 0, 0, 0],
            [0, 1, 1, 1, 0],
            [0, 1, 1, 1, 0],
            [0, 1, 1, 1, 0],
            [0, 1, 1, 1, 0],
            [0, 0, 0, 0, 0],
        ]
    ]
)
print(test_error.shape)

test_error_padded = np.pad(test_error[0], 1)
test_error2 = np.expand_dims(test_error_padded, (0, 1))

In [ ]:
test_error2.shape

In [ ]:
test_error

In [ ]:
weig = np.asarray([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]])

# ,
# [
#     [1, 0, -1],
#     [2, 0, -2],
#     [1, 0, -1]
# ]]

In [ ]:
weig.shape

In [ ]:
test_img = np.asarray(
    [
        [
            [
                [1, 1, 1, 2, 3],
                [1, 1, 1, 2, 3],
                [1, 1, 1, 2, 3],
                [2, 2, 2, 2, 3],
                [3, 3, 3, 3, 3],
                [4, 4, 4, 4, 4],
            ]
        ]
    ]
)

In [ ]:
conv = torch.nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3,
    bias=False,
    stride=1,
    padding_mode="zeros",
    padding=0,
)

x_tensor = torch.from_numpy(test_img.astype(np.float32))
x_tensor.requires_grad = True
conv.weight = torch.nn.Parameter(torch.from_numpy(weig.astype(np.float32)))
out = conv(x_tensor)

In [ ]:
loss = out.sum()
loss.backward()

print(conv.weight.grad)
print(x_tensor.grad)
print(loss)

In [ ]:
conv.weight

In [ ]:
dw = np.zeros_like(weig).astype(np.float32)
a_prev = np.pad(test_img, ((0, 0), (0, 0), (1, 1), (1, 1))).astype(np.float32)

# for b in range(out.shape[0]):
for c in range(test_img.shape[1]):
    for h in range(test_img.shape[2]):
        for w in range(test_img.shape[3]):
            dw[:, c] += a_prev[:, c, h : h + 3, w : w + 3] * 1.0

In [ ]:
dw

In [ ]:
error_conv = np.zeros((1, 1, 6, 5))
error_conv.shape

In [ ]:
test_error = np.asarray(
    [
        [
            [
                [0, 0, 0, 0, 0],
                [0, 1, 1, 1, 0],
                [0, 1, 1, 1, 0],
                [0, 1, 1, 1, 0],
                [0, 1, 1, 1, 0],
                [0, 0, 0, 0, 0],
            ]
        ]
    ]
)

# test_error = test_error[:, :, 1:-1, 1:-1]

da = np.zeros_like(test_img).astype(np.float32)
a_prev = np.pad(test_error, ((0, 0), (0, 0), (1, 1), (1, 1))).astype(np.float32)

# for b in range(out.shape[0]):
# for c in range(test_img.shape[1]):
for h in range(test_error.shape[2]):
    for w in range(test_error.shape[3]):
        da[:, :, h : h + 3, w : w + 3] += weig

In [ ]:
weig.shape

In [ ]:
da

In [ ]:
# for b in range(error_conv.shape[0]):
# for f in range(error_conv.shape[1]):
for h in range(error_conv.shape[2]):
    for w in range(error_conv.shape[3]):
        error_conv[:, :, h, w] = np.sum(test_error2[:, :, h : h + 3, w : w + 3] * weig)

In [ ]:
error_conv.shape

In [ ]:
error_conv

In [ ]:
import scipy

In [ ]:
scipy.signal.correlate2d(test_error[0][0], weig[0])

In [ ]:
test_error[0][0][1:-1, 1:-1] * error_conv[0][0]

In [ ]:
test_error2 = np.asarray(
    [
        [
            [0, 0, 0, 0, 0],
            [0, 1, 1, 1, 0],
            [0, 1, 1, 1, 0],
            [0, 1, 1, 1, 0],
            [0, 1, 1, 1, 0],
            [0, 0, 0, 0, 0],
        ]
    ]
)

test_error = test_error[1:-1, 1:-1]

test_inp = np.asarray(
    [
        [
            [1, 1, 1, 2, 3],
            [1, 1, 1, 2, 3],
            [1, 1, 1, 2, 3],
            [2, 2, 2, 2, 3],
            [3, 3, 3, 3, 3],
            [4, 4, 4, 4, 4],
        ],
        [
            [1, 1, 1, 2, 3],
            [1, 1, 1, 2, 3],
            [1, 1, 1, 2, 3],
            [2, 2, 2, 2, 3],
            [3, 3, 3, 3, 3],
            [4, 4, 4, 4, 4],
        ],
    ]
)

test_weig = np.asarray(
    [[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]
)

In [ ]:
test_grad = np.zeros((2, 3, 3))

In [ ]:
# for b in range(test_grad.shape[0]):
for h in range(test_grad.shape[1]):
    for w in range(test_grad.shape[2]):
        test_grad[:, h, w] = np.sum(
            test_inp[:, h : h + test_error.shape[0], w : w + test_error.shape[1]]
            * test_error,
            axis=(1, 2),
        )

In [ ]:
print(test_grad)

In [ ]:
test_error2 = np.expand_dims(test_error2, 0)

In [ ]:
test_error2.shape

In [ ]:
test_error2

In [ ]:
test_im_err = np.zeros((1, 1, 6, 5))

In [ ]:
test_weig.shape

In [ ]:
# for b in range(test_grad.shape[0]):
for h in range(6):
    for w in range(5):
        test_im_err[:, :, h, w] = np.sum(
            test_error2[:, :, h : h + 3, w : w + 3] * test_weig
        )

In [ ]:
test_im_err

In [ ]:
scipy.signal.correlate2d(test_error2[0][0], test_weig[0])